# QR Code 偵測

包含安裝 OpenCV 的指令。執行一次後即可跳過此步驟。首次安裝後需要重新啟動 kernel。

In [ ]:
import cv2
import kachaka_api
import numpy as np
from IPython.display import Image, clear_output, display

In [ ]:
client = kachaka_api.aio.KachakaApiClient()

執行以下程式碼後，請使用智慧型手機等裝置存取 https://pf-robotics.github.io/textcode/，輸入任意英數字，然後將顯示的 QR Code 靠近 Kachaka 的前置攝影機。

In [ ]:
qcd = cv2.QRCodeDetector()

async for image in client.front_camera_ros_compressed_image.stream():
    cv_image = cv2.imdecode(np.frombuffer(image.data, dtype=np.uint8), flags=1)
    decoded_info, corners, _ = qcd.detectAndDecode(cv_image)
    if corners is not None:
        cv_image = cv2.polylines(cv_image, corners.astype(int), True, (0, 0, 255), 2)
    if decoded_info != "":
        cv_image = cv2.putText(
            cv_image,
            decoded_info,
            corners[0][0].astype(int),
            cv2.FONT_HERSHEY_SIMPLEX,
            1,
            (0, 0, 255),
            2,
            cv2.LINE_AA,
        )
    _, ret = cv2.imencode(
        ".jpg",
        cv2.resize(cv_image, (int(cv_image.shape[1] / 2), int(cv_image.shape[0] / 2))),
    )
    clear_output(wait=True)
    display(Image(data=ret, format="jpeg"))